Importing libraries and defining global constants

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy

HBAR = 1 # Left in natural units, 6.582 * 10**(-22) [MeV s]
REDUCED_MASS = 10/11 * 931.5  # [Mev/c^2]
ORBITAL_QUANTUM_NUMBER = 0  # In units of hbar?
SUM_LIMIT = 5 # dictates number of dimensions of the eigenvalue problem
POTENTIAL_PARAMETERS = [40, 0.2]  # [fm^-2]
INITIAL_RANGE_PARAMETER = 0.01  # [fm^-2]
radius, E = sympy.symbols('radius E')

We now define the relevant Gaussian wavefunctions and potential for our system, as well as the kinetic energy operator and next range parameter

In [2]:
def gaussian_wave_function(radius, range_parameter, orbital_quantum_number=ORBITAL_QUANTUM_NUMBER):
    """
    Defines the form of the Gaussian wavefunctions used in the series expansion of the wavefunction in terms of the radius r,
    the orbital angular momentum, l, and a range parameter, α_i.

    Parameters
    ----------
    radius : float
    range_parameter : float characterising the gaussian

    Returns
    -------
    float the value of the gaussian wavefunction at the radius r
    """
    return radius**(orbital_quantum_number) * sympy.exp(- range_parameter * radius**2)


def gaussian_potential(radius, parameters=POTENTIAL_PARAMETERS):
    """
    Defines the spherically symmetric gaussian potential as a function of radius we use as a 'first guess'
    to make our integrals analytic. Given as V = V_0 * e**(-beta * radius**2)

    Parameters
    ----------
    radius : float
    parameters : 1d array containg the fixed parameters for our potential. In the order [V_0, beta] (see slides/supplimentary notes) 

    Returns
    -------
    float with the value of the potential at a radius r

    """
    return parameters[0] * sympy.exp(-1 * parameters[1] * radius**2)


def kinetic_energy_operator(radius, wavefunction, range_parameter, hbar=HBAR, reduced_mass=REDUCED_MASS,
                            orbital_quantum_number=ORBITAL_QUANTUM_NUMBER):
    """
    Defines the kinetic energy operator for our system: '-\frac{\hbar^2}{2\mu}\frac{d^2}{dr^2} + \frac{l(l+1\hbar^2}{2\mu r^2}'

    Parameters
    ----------
    radius : float
    wavefunction : sympy expression describing the wavefunction of the system
    range_parameter : float characterising the gaussian

    Returns
    -------
    sympy expression describing the kinetic energy term in the eigenvalue problem

    """
    return - hbar**2 / (2 * reduced_mass) * sympy.diff(wavefunction(radius, range_parameter), radius, radius) + (
        orbital_quantum_number * (orbital_quantum_number + 1) * hbar**2) / (2 * reduced_mass * radius**2)


def next_range_parameter(i, initial_range_parameter=INITIAL_RANGE_PARAMETER, some_number=2):
    """
    Finds the next range parameter given the previous and initial range parameters.
    Currently using a simple geometric series to determine range parameters.
    Chose geometric basis parameters $\alpha_i = \alpha_1a^{i-1}$ with initial parameters $\alpha_1 = 0.01, a=2$

    Parameters
    ----------
    i : int detailing the iteration number

    Returns
    -------
    new_range_parameter: float

    """
    new_range_parameter = initial_range_parameter * some_number**(i-1)
    return new_range_parameter

We now define our matrix elements

In [3]:
def N_ij(radius, wavefunction_1, wavefunction_2, range_parameter_1, range_parameter_2):
    """
    Finds an expression for the overlap term in the eigenvalue problem.
    Evaluates the expression  N_{ij} = \int_0^{\infty}\phi_i(r)\phi_j(r)dr

    Parameters
    ----------
    radius : float
    wavefunction_1 : an expression for one of the two wavefunctions in the system
    wavefunction_2 : an expression for one of the two wavefunctions in the system
    range_parameter_1 : float, range parameter associated with the first wavefunction
    range_parameter_2 : float, range parameter associated with the second wavefunction

    Returns
    -------
    sympy expression describing the overlap term in the eigenvalue problem

    """
    return sympy.integrate(wavefunction_1(radius, range_parameter_1) * wavefunction_2(radius, range_parameter_2), (radius, 0, sympy.oo))


def T_ij(radius, wavefunction_1, wavefunction_2, range_parameter_1, range_parameter_2):
    """
    Finds an expression for the kinetic energy term in the eigenvalue problem.
    Evaluates the expression  T_{ij} = \int_0^{\infty}\phi_i(r)(-\frac{\hbar^2}{2\mu}\frac{d^2}{dr^2} + \frac{l(l+1\hbar^2}{2\mu r^2})\phi_j(r)dr

    Parameters
    ----------
    radius : float
    wavefunction_1 : an expression for one of the two wavefunctions in the system
    wavefunction_2 : an expression for one of the two wavefunctions in the system
    range_parameter_1 : float, range parameter associated with the first wavefunction
    range_parameter_2 : float, range parameter associated with the second wavefunction

    Returns
    -------
    sympy expression describing the kinetic energy term in the eigenvalue problem

    """
    return sympy.integrate(wavefunction_1(radius, range_parameter_1) * kinetic_energy_operator(radius, wavefunction_2, range_parameter_2),
                           (radius, 0, sympy.oo))


def V_ij(radius, wavefunction_1, wavefunction_2, range_parameter_1, range_parameter_2, potential_function):
    """
    Finds an expression for the potential energy term in the eigenvalue problem.
    Evaluates the expression  V_{ij} = \int_0^{\infty}\phi_i(r)V(r)\phi_j(r)dr

    Parameters
    ----------
    radius : float
    wavefunction_1 : an expression for one of the two wavefunctions in the system
    wavefunction_2 : an expression for one of the two wavefunctions in the system
    range_parameter_1 : float, range parameter associated with the first wavefunction
    range_parameter_2 : float, range parameter associated with the second wavefunction
    potential_function : sympy expression detailing the form of the potential

    Returns
    -------
    sympy expression describing the potential energy term in the eigenvalue problem

    """
    return sympy.integrate(wavefunction_1(radius, range_parameter_1) * potential_function(radius) * wavefunction_2(radius, range_parameter_2),
                           (radius, 0, sympy.oo))


We now need to create the H_{ij} and N_{ij} matrices and solve the characteristic equation for a GENERALISED eigenvalue problem, and save this as a variable.

In [4]:
# generate matrices for the hamiltonian and overlap term for the eigenvalue problem
h_matrix = sympy.zeros(SUM_LIMIT - 1, SUM_LIMIT - 1)
n_matrix = sympy.zeros(SUM_LIMIT - 1, SUM_LIMIT - 1)
i = 1
j = 1
i_range_parameter = INITIAL_RANGE_PARAMETER
j_range_parameter = INITIAL_RANGE_PARAMETER

while i < SUM_LIMIT:
    while j < SUM_LIMIT:
        # iterate over every matrix element to find H_{ij} and N_{ij}
        # H_{ij} = T_{ij} + V_{ij}
        h_matrix[i-1, j-1] = T_ij(radius, gaussian_wave_function, gaussian_wave_function, i_range_parameter, j_range_parameter) + V_ij(
            radius, gaussian_wave_function, gaussian_wave_function, i_range_parameter, j_range_parameter, gaussian_potential)
    
        n_matrix[i-1, j-1] =N_ij(radius, gaussian_wave_function, gaussian_wave_function, i_range_parameter, j_range_parameter)

        # update matrix position and associated range parameter accordingly
        j += 1
        j_range_parameter = next_range_parameter(j)
    i += 1
    i_range_parameter = next_range_parameter(i)
    j = 1
    j_range_parameter = INITIAL_RANGE_PARAMETER

# returns system of expressions, set equal to zero and solve generalised eigenvalue problem

We now seek to solve for the generalised eigenvalues for our system of equations

In [5]:
generalised_eigenvalues = sympy.solve(sympy.det(h_matrix - E * n_matrix), E)
print(generalised_eigenvalues)

[0.000354787171881519, 0.101983159431602, 4.59001411726263, 31.4574568592418]


We now want the generalised eigenvectors - will work on this next session